In [1]:
import torch
from torch.utils.data import DataLoader

import matplotlib.pyplot as plt
import time

import datasets.mvtec as mvtec

from feature_extraction import Resnet18Features, WideResnet50Features, extractEmbeddingVectors, getIndices
# from distribution_fitting import getMeanCov
from distance_calculation import calculateScoreMaps
# from padim_utils import saveFeatures, loadFeatures
from __init__ import AnomalyDetector, imageAnomalyDetection


In [22]:
use_cuda = torch.cuda.is_available()
device = torch.device('cuda' if use_cuda else 'cpu')
# device = torch.device('cpu')
device

device(type='cuda')

In [29]:
class_name = 'down'
data_path = '../data/aligned_pucks/'
dataset = mvtec.MVTecDataset(data_path, class_name=class_name, is_train=True)
dataloader = DataLoader(dataset, batch_size=32, pin_memory=True)
len(dataset)
x = next(iter(dataloader))[0][0].unsqueeze(0)
# x = next(iter(dataloader))[0][:20]
x.shape

torch.Size([1, 3, 224, 224])

In [30]:
features_model = Resnet18Features(device)
# features_model = WideResnet50Features(device)

In [31]:
tic = time.perf_counter()
mean = torch.load('./distributions/up_mean_test.pkl')
cov_inv = torch.load('./distributions/up_cov_inv_test.pkl')
mean = mean.to(device)
cov_inv = cov_inv.to(device)

print(time.perf_counter()-tic)


0.06665721599711105


In [32]:
embedding_vectors = extractEmbeddingVectors(features_model, x, device)
idx = getIndices(100, 448, device)  
embedding_vectors = torch.index_select(embedding_vectors, 1, idx)

In [33]:
tic = time.perf_counter()
score_maps = calculateScoreMaps(mean, cov_inv, embedding_vectors, device=device, do_gaussian_filter=True)
print(time.perf_counter()-tic)


0.007298326003365219


In [34]:
print(score_maps.shape)
score_maps[0]

torch.Size([56, 56])


tensor([11.5460, 11.7068, 12.0087, 12.4149, 12.8779, 13.3466, 13.7732, 14.1205,
        14.3656, 14.5036, 14.5473, 14.5242, 14.4709, 14.4250, 14.4165, 14.4614,
        14.5579, 14.6871, 14.8183, 14.9147, 14.9416, 14.8716, 14.6904, 14.3984,
        14.0118, 13.5601, 13.0825, 12.6225, 12.2220, 11.9169, 11.7330, 11.6845,
        11.7731, 11.9885, 12.3094, 12.7057, 13.1412, 13.5769, 13.9745, 14.3007,
        14.5304, 14.6500, 14.6588, 14.5691, 14.4030, 14.1869, 13.9461, 13.6991,
        13.4561, 13.2201, 12.9913, 12.7718, 12.5689, 12.3955, 12.2674, 12.1990],
       device='cuda:0')